<a href="https://colab.research.google.com/github/krushcev/trycrypto/blob/master/binanceccxt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ccxt

In [8]:
pip install pandas

In [9]:
import ccxt
import pandas as pd

In [20]:
exchange = ccxt.binance()
markets = exchange.load_markets()
df = {}
errp = 0

In [22]:
def is_consolidating(df, percentage=2):
    recent_candlesticks = df[-15:]
    last_close = df[-1:]['close'].values
    
    max_close = recent_candlesticks['close'].max()
    min_close = recent_candlesticks['close'].min()

    threshold = 1 - (percentage / 100)
    if min_close > (max_close * threshold) and last_close != 50:
        return True        

    return False

def is_on_weakness(df):
    recent_candlesticks = df[-15:]
    last_close = df[-1:]['close'].values
    last_xdays = df[-2:]['close'].values

    min_close = recent_candlesticks['close'].min()
    if (last_xdays <= min_close).all() and (last_close != 50).all():
        return True
    
    return False

def is_breaking_out(df, percentage=10):
    last_close = df[-1:]['close'].values

    if is_consolidating(df[:-1], percentage=percentage):
        recent_closes = df[-7:-1]

        if last_close > recent_closes['close'].max():
            return True

In [ ]:
for market in markets:
  if '/BTC' in market:
    try:
      ohlc = exchange.fetch_ohlcv(market, timeframe='1d', limit=30)
      df[market] = pd.DataFrame(ohlc, columns=['timestamp','open','high','low','close','volume'])
      df[market]['timestamp'] = pd.to_datetime(df[market]['timestamp'], unit='ms')
    except:
      errp +=1
    else:
      if is_breaking_out(df=df[market]):
        print("{} is breaking out!".format(market))
        print(df[market][-1:])
      if is_consolidating(df=df[market], percentage=10):
        print("{} is my new watchlist".format(market))
        print(df[market][-1:])
      if is_on_weakness(df=df[market]):
        print("{} is breaking out!".format(market))
        print(df[market][-1:])

print("Error counts : {}".format(errp))